This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([177.28302038, 150.37102595, 202.0924573 , ...,   0.        ,
        23.809066  ,   0.        ]), array([218.49190694, 158.37366148, 143.26116819, ...,  67.13015962,
        48.82057995,   4.40653003]), OrderedDict([('R2', 0.7625207437033727), ('root_mean_squared_error', 47.59156345050221), ('mean_absolute_error', 32.27457536932615), ('rmse_over_stdev', 0.4918620510346152)]), None)
test_quad = (array([2.18800571e+02, 2.39214774e+02, 2.41537703e+02, 2.19470531e+02,
       2.01402113e+02, 3.88395179e+02, 2.44558934e+02, 7.37475037e+01,
       1.40791488e+02, 9.96717956e+01, 2.41991216e+02, 2.14252916e+02,
       2.00047469e+02, 1.45936466e+02, 1.13270942e+02, 8.53211606e+01,
       1.01679243e+02, 2.33371776e+02, 1.81776188e+02, 1.91511615e+02,
       1.58791749e+02, 1.79995444e+02, 1.25772961e+02, 3.50019597e+02,
       3.06666897e+02, 2.29722452e+02, 1.72040539e+02, 1.03964994e+02,
       8.84608475e+01, 1.87427677e+02, 1.65793652e+02, 1.74631224e+02,
       1.68464956e+02, 1.31350152e+02, 3.79332462e+01, 2.16105345e+01,
       9.24493164e+01, 2.89476692e+01, 1.71608093e+02, 9.51442246e+01,
       2.36177315e+02, 2.54265050e+02, 2.91384616e+02, 5.15805330e+02,
       1.78235739e+02, 8.07377667e+01, 8.42025056e+01, 8.95029615e+01,
       1.36316077e+02, 1.03801111e+02, 9.24561079e+01, 0.00000000e+00,
       4.93432857e+01, 1.58218632e+02, 5.66738367e+01, 2.73135364e+01,
       1.56215250e+01, 1.01797990e+02, 5.64799569e+01, 1.77441804e+02,
       5.18644462e+01, 1.12003551e+02, 1.39599684e+02, 1.19778837e+02,
       2.18946397e+02, 2.53560446e+02, 2.02689680e+02, 1.50472708e+02,
       1.17650530e+02, 7.08357719e+01, 0.00000000e+00, 3.31384554e+01,
       4.09146349e+01, 5.57153253e+01, 2.58389489e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 9.45062097e+01, 1.69769380e+02, 7.39741468e+01,
       1.26328149e+01, 1.54429064e+01, 1.65750556e+01, 0.00000000e+00,
       3.34361123e+01, 7.74558848e+00, 0.00000000e+00, 3.21439514e+01,
       0.00000000e+00, 4.42073704e-01, 1.78001455e+02, 0.00000000e+00,
       1.33559658e+02, 1.40198319e+02, 1.50508186e+02, 1.14247314e+02,
       9.57636772e+01, 1.02514386e+02, 0.00000000e+00, 3.04689791e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.43690177e+01,
       8.99135404e+01, 0.00000000e+00, 5.53782777e+01, 0.00000000e+00,
       6.29442201e+01, 2.88123722e+01, 2.41335092e+01, 1.22038739e+02,
       4.48563810e+01, 2.61695184e+01, 2.91224504e+01, 3.66852424e+01,
       2.82257425e+01, 7.30016409e+01, 1.04542615e+02, 3.62570480e+01,
       5.86508558e+01, 3.50982856e+00, 1.26886312e+02, 2.78984665e+02,
       2.83906453e+02, 1.38892855e+02, 8.77843631e+01, 1.22311356e+02,
       0.00000000e+00, 0.00000000e+00, 1.30597392e+02, 3.29412986e+01,
       2.63839575e+02, 4.45414483e+01, 5.20286768e+01, 2.29064683e+02,
       1.34506442e+02, 2.99692485e+02, 1.99684569e+02, 4.96917625e+01,
       0.00000000e+00, 4.78013760e+01, 2.23740347e+01, 3.61695636e+01,
       2.62303388e+01, 3.36549838e+01, 1.05110827e+01, 0.00000000e+00,
       1.21655324e+01, 0.00000000e+00, 7.81817223e+01, 4.48488874e+01,
       4.49236078e+01, 4.71419734e+01, 3.08078053e+01, 1.70719435e+01,
       0.00000000e+00, 2.34800709e+02, 7.83502295e+01, 6.06316356e+01,
       1.10052006e+02, 1.57158326e+02, 5.02984410e+01, 3.32733667e+01,
       0.00000000e+00, 3.95429854e+01, 4.16645268e+01, 2.82496436e+01,
       3.13514481e+01, 5.20549201e+01, 3.92079232e+01, 2.14364169e+02,
       1.24729865e+02, 1.28386129e+02, 9.46538849e+01, 1.76733958e+01,
       1.03527711e+01, 3.36737583e+01, 1.15904985e+01, 6.67613331e+01,
       4.23440436e+01, 1.03266329e+02, 8.58908577e+01, 5.43550612e+00,
       3.57280662e+01, 0.00000000e+00, 9.36490231e+00, 6.90074178e+01,
       0.00000000e+00, 0.00000000e+00, 1.33030325e+02, 1.42333954e+02,
       1.68517827e+02, 1.36431444e+02, 1.82075421e+02, 2.45692439e+02,
       2.17252824e+02, 1.89093084e+02, 2.34660651e+02, 1.57972803e+02,
       2.13314660e+02, 3.78453371e+02, 2.87661839e+02, 3.42040019e+02,
       3.36224697e+02, 1.06514784e+02, 1.34673117e+02, 1.13869162e+02,
       1.81295285e+02, 2.49724729e+02, 1.31761462e+02, 1.86026530e+02,
       1.26700822e+02, 1.63799175e+02, 1.69625113e+02, 1.52280199e+02,
       1.29342617e+02, 1.22431670e+02, 1.77752168e+02, 1.69573331e+02,
       2.92536084e+02, 2.67340580e+02, 2.62896301e+02, 2.22246511e+02,
       2.06364164e+02, 3.20529825e+02, 3.18399908e+02, 2.01321423e+02,
       2.32699819e+02, 1.29319114e+02, 1.80660958e+02, 1.30558998e+02,
       8.23163755e+01, 9.69915119e+01, 8.86267599e+01, 2.47684975e+02,
       1.61293351e+02, 2.08659157e+02, 2.47005095e+02, 9.69267962e+01,
       1.93969856e+02, 1.87885255e+02, 1.88513164e+02, 1.92010418e+02,
       2.13333228e+02, 3.54737752e+02, 3.29042834e+02, 1.44889042e+02,
       2.47474247e+02, 8.54936441e+01, 1.24842572e+02, 1.44154498e+02,
       6.84009027e+01, 5.59498437e+01, 1.70005981e+02, 1.38575932e+02,
       1.48873887e+02, 1.09381016e+02, 9.73721500e+01, 5.55045542e+01,
       1.29155595e+02, 4.52932141e+01, 5.68526124e+01, 1.43519281e+02,
       1.99578096e+02, 2.33383666e+02, 2.72871580e+02, 1.84420825e+02,
       1.32211588e+02, 3.04851246e+02, 2.28169699e+02, 1.73746536e+02,
       2.04554894e+02, 2.04542101e+02, 2.09325439e+02, 1.42416121e+02,
       8.21190201e+01, 9.69622803e+01, 5.74165434e+01, 5.19883620e+01,
       1.51089123e+02, 9.54271066e+01, 6.28478449e+01, 1.89038755e+01,
       2.66189733e+01, 1.74303139e+01, 3.31481761e+01, 1.27664645e+02,
       2.20249128e+01, 4.28892788e+01, 5.66837962e+01, 7.59835817e+01,
       4.29043523e+01, 7.75933817e+01, 1.73730028e+02, 1.62590223e+02,
       1.76433607e+02, 1.73385595e+02, 1.88271436e+02, 2.10616630e+02,
       2.00391998e+02, 1.22370956e+02, 9.06089707e+01, 1.13060644e+02,
       1.83271284e+02, 1.17249168e+02, 1.26798490e+02, 2.96630230e+02,
       1.85661694e+02, 1.64660391e+02, 3.97239965e+02, 1.12287386e+02,
       1.09297300e+02, 9.87844184e+01, 1.38582903e+02, 1.40364781e+02,
       1.36808521e+02, 3.35321293e+02, 8.86858956e+01, 7.98648319e+01,
       6.97752946e+01, 2.63543371e+02, 1.07783426e+02, 5.65113296e+01,
       3.33198729e+01, 1.83475519e+02, 2.86171985e+01, 4.69172371e+01,
       2.71330052e+01, 1.18947119e+02, 1.51299348e+01, 3.73225941e+01,
       5.97413320e+01, 1.09582964e+02, 3.97988019e+01, 5.04471564e+01,
       7.14796192e+01, 7.26752642e+01, 7.84670662e+01, 1.34244899e+02,
       4.06890701e+01, 6.16998797e+01, 9.38114347e+01, 3.41670362e+02,
       3.87496682e+01, 0.00000000e+00, 9.82564416e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 6.41409237e+01, 1.98105436e+02,
       1.13483054e+02, 6.23247673e+01, 5.63192894e+01, 5.41514922e+01,
       5.32856309e+01, 4.94793928e+01, 7.26725204e+01, 7.32347672e+01,
       1.01755733e+02, 2.12813897e+01, 3.38853640e+01, 9.31924847e+00,
       1.24011708e+01, 3.23684503e+01, 4.56348093e+01, 3.17633706e+01,
       2.99157066e+01, 3.77846853e+01, 1.72121127e+01, 2.01387280e+01,
       7.56574945e+01, 7.65032664e+01, 7.72377141e+01, 6.05410832e+01,
       1.66797820e+02, 7.05986967e+01, 1.21693503e+02, 4.54995709e+01,
       0.00000000e+00, 4.44093613e+01, 4.82400402e+01, 3.39806022e+01,
       3.91274706e+01, 7.33023683e+01, 8.25826337e+01, 7.09430276e+01,
       3.76093386e+01, 3.62234796e+01, 9.01656399e+01, 4.11379073e+01,
       2.72516127e+01, 3.15042600e+01, 3.07458097e+01, 1.20865840e+02,
       4.59371875e+01, 2.93690973e+01, 9.40057842e+01, 6.65012486e+01,
       4.54076116e+01, 5.19044272e+01, 3.87785474e+01, 5.24254612e+01,
       4.50623816e+01, 8.97135102e+01, 1.25245007e+02, 5.97432500e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.88957798e+01, 0.00000000e+00, 0.00000000e+00,
       4.72079029e+01]), array([190.31006537, 225.96890127, 162.93271385, 154.06887399,
       151.96269007, 273.732219  , 212.10310946, 138.35511192,
       112.73210861,  86.78179531, 247.63465254, 147.16911865,
       149.94510528, 137.29526552, 136.75464259, 100.35045099,
        92.22025578, 232.79564154, 171.22970082, 291.43658628,
       151.62354036, 150.91421436, 143.66045609, 242.10404226,
       267.49859505, 316.26351345, 193.74227663, 106.74235832,
        85.4196551 , 148.80561093, 151.58512525, 147.08077793,
       196.02173355,  28.32704865,  88.78013868,  28.23790509,
        91.63903771,  45.54424914, 127.69659012,  80.32859548,
        85.76819945,  87.64310448, 193.10182142, 509.94537352,
        83.4228358 , 129.05105773, 125.28366659, 108.34441892,
       140.84422391, 135.4346509 ,  85.02041234, 104.29525227,
       195.92586257, 115.08481155, 143.27115687,  77.06134967,
       106.32978154, 143.2901173 ,  60.96468024,  78.36695873,
        65.31785766, 122.3529941 , 138.51200477, 139.67293224,
       164.15363414, 220.82244079, 168.2218547 , 153.4316764 ,
        96.02719439,  78.65591276,   6.14073499,  76.18337433,
        59.20102592,  82.07971081,  59.89210155,  -3.40358094,
         6.79934582,  37.40910176,  32.96139774,  24.01026436,
        24.35412117,  59.76396725,  35.05694041,  16.8589657 ,
        24.5526336 ,  25.52541254,  21.88445442,  42.52723145,
        46.89618779,  92.42156665, 145.90317532,  82.62484821,
        33.0177533 ,  39.72195581,  65.01431996,  36.27282904,
        42.6130736 ,  44.58336869,  -2.24703185,  50.6468284 ,
        28.84582844,  64.38292421, 170.91768313,  13.53629712,
       106.20159193, 103.80207786, 106.43929438,  96.11338849,
        86.38644646,  87.26376865,  20.01984235,  42.1324558 ,
        14.42686619,  14.80619743,  15.74638193,  49.89622835,
        93.99871929,  35.60360849,  36.05927583,  26.82041953,
        32.77781787,  47.62718344,  21.72839476,  74.7871835 ,
        17.99080873,   8.49810018,  10.92807246,  30.57667152,
        36.27988252,  63.5743238 ,  59.83720905,   5.31871838,
        15.95701666,  33.80121802, 158.49803795, 330.24567551,
       232.58462996,  92.18487941, 125.7717719 , 114.23394067,
        93.79713934,   0.        ,  80.25623702,  68.72490929,
       287.14266998, 129.18000057,  32.20499401, 106.40399237,
        66.47867619, 211.86918872,  76.38070454,  66.88482866,
         7.45439608,  53.18926053,  42.67223043,  42.69966506,
        44.48799451,  44.8751601 ,  23.25585951,  12.14336248,
        39.48613349,  22.2215469 ,  70.78841786,  46.26491232,
        46.83668005,  47.950501  ,  36.75876122,  27.91556311,
        -4.17008923, 160.22342956,  85.23249936, 101.09239593,
        66.02130115,  71.23336537,  69.65200099,  63.75862708,
        -2.80922219,  51.40845101,  55.05178677,  64.3873969 ,
        41.03599482,  35.41989897,  50.08917496, 266.98727562,
       109.93882274, 110.05876872,  90.8593222 ,  24.30027701,
        28.10377619,  25.65298759,  34.18221169,  44.08457614,
        41.76404352,  37.9385526 ,  43.27991951,  28.79301859,
         3.17660904,  15.85310227,  42.00601231,  12.35592084,
        14.26355583,  13.85946358, 213.39445314, 179.3680015 ,
       142.52109003,  19.38562096, 161.4044429 , 310.19263592,
       305.20136221, 194.15554552, 223.83838258, 118.35324943,
       153.76469997, 308.83305725, 265.85047628, 276.0134192 ,
       257.61523563, 142.71556054, 210.6510166 , 158.82918555,
       254.88067904, 164.55523692, 103.27351341, 199.15230613,
       186.05694296, 182.91728612, 202.8056389 ,  98.98692803,
       124.93006433, 120.49942312, 154.19585641, 140.32290639,
       145.44397733, 309.10122633, 253.97194901, 268.82302666,
       227.8107564 , 229.0029697 , 268.53210568, 176.63913751,
       215.14550614,  87.74866009, 205.49058395, 163.97137708,
        82.84356976,  81.75902833, 131.55294844, 241.2401896 ,
       167.57632012, 224.17866693, 261.88052739,  93.25645177,
        96.84663603,  89.48185845,  75.42188316, 138.34274123,
       161.36371572, 310.23762722, 308.17980676, 157.81396547,
       204.93731891,  90.82862018, 184.89176956, 171.72721625,
        83.75690158,  84.04774674,  85.95718291, 124.26432745,
       205.71677396, 138.92608065, 145.51574534,  90.62501848,
       111.97210866,  69.50651967,  66.12152854,  73.27568899,
       172.80695798, 139.01097998, 309.40450467, 222.6920204 ,
       167.53808879, 238.50310292, 278.0062831 , 175.69780885,
       201.77510189, 222.4294716 , 158.25198268, 139.20274151,
       123.67644812, 125.67845941, 112.87045956, 114.14584339,
       147.94291067, 121.29296055,  71.18241149,  77.67422466,
        71.89505883,  22.92892517,  25.00129473, 138.91410275,
        34.37367086,  42.61376622,  43.0477209 ,  71.23594543,
        23.81795586,  43.0323279 , 181.71926405, 167.73130807,
       174.89516072, 158.90990958, 171.46538427, 174.3206443 ,
       188.55000618, 123.69171345, 126.50807041, 115.19907111,
       199.32768553, 124.69875822, 146.02268591, 310.2562208 ,
       124.2025793 , 131.18828372, 312.26975728, 104.54497468,
       102.0237248 , 101.78892258, 109.97043732, 113.44366635,
       126.50663769, 312.28235559, 111.18604225,  98.24394466,
        98.77163573, 311.47355253, 145.16608271,  63.22051544,
        35.49573555,  99.48067435,  36.85892809,  49.40520194,
        39.52181894,  83.88832642,  45.79563109,  41.42577844,
        58.2918433 , 101.97854087,  86.32267497,  96.25146831,
        97.80978565,  93.59403384, 101.71878067, 105.73925497,
        84.75370262, 104.00332855,  79.31736742, 180.42904751,
        55.49836203,   8.62472049,  55.28576086,  22.97304518,
         6.27938337,   6.51259016,  58.03115499, 147.52922809,
        78.90453478,  75.12113308,  40.70268938,  49.69842518,
        43.39115595,  43.89723624,  76.4673982 ,  57.79529879,
        47.77452003,  -0.82735138,  11.5292014 ,  11.65173694,
        37.97225288,  26.08690201,  56.28650392,  28.30600666,
        26.32107973,  36.15658738,  26.48498955,  29.1643135 ,
        49.19825656,  51.83144933,  44.92226988,  45.53975071,
        63.75349284,  52.38226235, 128.33543098,  30.70480029,
        -2.41969044,  39.35629836,  51.63268926,  38.96314006,
        31.0337008 ,  89.44822703,  83.96662346,  83.83283867,
        64.78112675,  57.47201787,  82.84501558,  27.71701216,
        25.3739043 ,  21.00865899,  23.53397787,  89.78930484,
        58.00693835,   9.21696434,  57.08107197,  33.7974346 ,
        33.90852966,  45.44641518,  30.73395536,  44.76704779,
        27.15022285,  62.32851783,  73.21003978,  17.55490225,
         7.93268951,  15.67222943,  16.50671609,   8.90386518,
        29.71196128,  45.16412504,  33.54985704,  -3.02953029,
        61.28474602]), OrderedDict([('R2', 0.7833798140469362), ('root_mean_squared_error', 41.87814445528851), ('mean_absolute_error', 30.229951584330216), ('rmse_over_stdev', 0.4328134764205723)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/learning_curves/learning/StandardScaler/SequentialFeatureSelector/KernelRidge/RepeatedKFold/split_0'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
